In [29]:
import gymnasium as gym
import gymnasium_robotics

# PyTorch
import torch
import os

# from collections import deque
import numpy as np
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback

In [2]:
env_id = 'FrankaKitchen-v1'
task = 'kettle'
gym.register_envs(gymnasium_robotics)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
n_actions = 9

In [5]:
flat_dim = 73
obs_low = np.full((flat_dim,), -1e10, dtype=np.float32)
obs_high = np.full((flat_dim,), 1e10, dtype=np.float32)

class FlattenDictWrapper(gym.ObservationWrapper):    
    def __init__(self, env):
        super().__init__(env)
        self.keys = env.observation_space.spaces.keys()
        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, shape=(flat_dim,), dtype=np.float32)

    def observation(self, observation):
        achieved = observation['achieved_goal'][task].astype(np.float32)
        desired = observation['desired_goal'][task].astype(np.float32)
        obs = observation['observation'].astype(np.float32)

        flat_obs = np.concatenate([achieved, desired, obs], dtype=np.float32)
        return flat_obs

In [26]:
def make_env():
    env = gym.make(env_id, render_mode=None, tasks_to_complete=[task])  # Or your actual task
    env = FlattenDictWrapper(env)
    return env

n_training_envs = 64
env = DummyVecEnv([make_env]*n_training_envs)
eval_env = DummyVecEnv([make_env])

In [38]:
max_timesteps = 250000
run_name = f"ddpg_{max_timesteps}_"+task

In [39]:
eval_log_dir = os.path.join("eval_logs", run_name)
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                              log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                              n_eval_episodes=5, deterministic=True,
                              render=False)

In [40]:

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1, device=device, buffer_size=5000000)


Using cuda:0 device


/home/pc/repos/rl/rl-value-based/venv/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 3.14GB > 1.35GB
  warnings.warn(


In [ ]:
model.learn(total_timesteps=max_timesteps, log_interval=10, callback=eval_callback)

In [ ]:
model.save(run_name)

In [33]:
for i in range(10):
	env_eval = make_env()
	obs, _ = env_eval.reset()
	done = False
	ep_reward = 0

	while not done:
		action, _ = model.predict(obs, deterministic=True)
		obs, reward, terminated, truncated, _ = env_eval.step(action)
		done = terminated or truncated
		ep_reward += reward
	print(f"Episode reward: {ep_reward}")

Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0


In [ ]:
print("Action space:", env_eval.action_space)
print("Model action space:", model.action_space)

Action space: Box(-1.0, 1.0, (9,), float64)
Model action space: Box(-1.0, 1.0, (9,), float64)
